# Column Generation

## Cutting stock
\begin{align*}
\min \sum_{i=1}^m y_i\\
\sum_{j=1}^n l_j x_{ij} &\leq L y_i \quad \forall i \in \{1,\ldots, m\}\\
\sum_{i=1}^m x_{ij} &\geq d_j \quad \forall j \in \{1,\ldots, n\}\\
x_{ij} &\in \mathbb{Z}^+ \quad \forall j \in \{1,\ldots, n\}, \forall i \in \{1,\ldots, m\}\\
y_i &\in \{0,1\} \quad \forall i \in \{1,\ldots, m\}
\end{align*}

In questo problema è necessario tagliare delle assi (tutte di pari lunghezza) in modo da minimizzare il numero di assi utilizzate e soddisfare la richiesta totale.

Il modello presenta un elevato grado di simmetria, cioè esistono tante soluzioni ottime equivalenti. Questo mette in crisi i risolutori ad albero.

In [1]:
import random
random.seed(42) # For reproducibility 

In [2]:
from pprint import pprint
import math
import numpy as np

In [3]:
import scipy.linalg as linalg 
from pyscipopt import Model, quicksum

In [4]:
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [5]:
def print_results(model:Model):
    """Prints the given model solution
        model: the solved model to be printed
    """
    name = model.getProbName()
    if model.getPrimalbound() == 1e+20:
        print("A feaseable solution does not exist in model %s."%name)
    else:
        print("In model %s the optimal solution value is : %s"%(name, model.getObjVal()))

In [6]:
def cutting_stock(l:list, d:list, L:int, m:int):
    """Returns a cutting stock model
        l: the list of products lenghts
        d: the products demands list
        L: the maximum lenght of a paling
        m: the maximum number of palings
    """
    model = Model("Cutting Stock")
    x,y = {},{}
    n = len(d)
    
    # Adding the variables to the model
    for i in range(m):
        y[i] = model.addVar(vtype="B", name="y(%s)"%i)
        for j in range(n):
            x[i,j] = model.addVar(vtype="I", name="x(%s,%s)"%(i,j))
            #Since I don't know how to force R+, I add a constraint:
            model.addCons(x[i,j] >= 0, "Positive(%s,%s)"%(i,j))
            
    #Adding length constraints
    for i in range(m):
        model.addCons(quicksum(l[j]*x[i,j] for j in range(n)) <= L*y[i], "Length(%s)"%i)
        
    #Adding demand constraints
    for j in range(n):
        model.addCons(quicksum(x[i,j] for i in range(m)) >= d[j], "Demand(%s)"%j)
        
    # Setting objective function
    model.setObjective(quicksum(y[i] for i in range(m)), "minimize")
    
    model.data = x,y
    return model

## Formulazione estesa

\begin{align*}
\min \sum_{h=1}^H z_h\\
\sum_{h=1}^H a_{jh} z_h &\geq d_j \quad \forall j \in \{1,\ldots, n\}\\
z_h &\in \{0,1\} \quad \forall h \in \{1,\ldots, H\}
\end{align*}


In [7]:
def extended_formulation(A:list, d:list):
    """Generates a reduced master problem model
        A: matrix of cut patterns
        d: list of products demands
    """
    model = Model("Cutting Stock Extended Formulation")
    z = {}
    n = len(d)
    H = len(A)
    
    # Adding the variables to the model
    for h in range(H):
        z[h] = model.addVar(vtype="B", name="z(%s)"%(h))
        
    #Adding demand constraints
    for j in range(n):
        model.addCons(quicksum(A[h][j]*z[h] for h in range(H)) >= d[j], "Demand(%s)"%j)
        
    # Setting objective function
    model.setObjective(quicksum(z[h] for h in range(H)), "minimize")
    
    model.data = z
    return model

## Master Problem

\begin{align*}
\min \sum_{h=1}^H z_h\\
\sum_{h=1}^H a_{jh} z_h &\geq d_j \quad \forall j \in \{1,\ldots, n\}\\
z_h &\in [0,1] \quad \forall h \in \{1,\ldots, H\}
\end{align*}


In [8]:
def starting_cuts_pattern_set(l:list, d:list, L:int):
    """Generates a starting cut pattern set
        l: list of products lengths
        d: list of demands for each product
        L: maximum paling length 
    """
    patterns = [] # patterns
    m = len(l)
    # Generate initial patterns with one size for each item width
    for (i,width) in enumerate(l):
        pattern = [0]*m  # vector of number of orders to be packed into one roll (bin)
        quantity = int(L/width)
        pattern[i] = quantity
        for j in range(math.ceil(d[i]/quantity)):
            patterns.append(pattern)
    return patterns

## Master Problem Ridotto

\begin{align*}
\min \sum_{h=1}^{\bar{H}} z_h\\
\sum_{h=1}^{\bar{H}} a_{hj} z_h &\geq d_j \quad \forall j \in \{1,\ldots, n\}\\
z_h &\in [0,1] \quad \forall h \in \{1,\ldots, \bar{H}\}
\end{align*}


In [9]:
def reduced_master_problem(A:list, d:list):
    """Generates a reduced master problem model
        A: matrix of cut patterns
        d: list of products demands
    """
    model = Model("Reduced Master Problem")
    z = {}
    n = len(d)
    H = len(A)
    
    # Adding the variables to the model
    for h in range(H):
        z[h] = model.addVar(vtype="C", name="z(%s)"%(h))
        model.addCons(z[h] >= 0, "Zero(%s)"%(h))
        model.addCons(z[h] <= 1, "One(%s)"%(h))
        
    #Adding demand constraints
    for j in range(n):
        model.addCons(quicksum(A[h][j]*z[h] for h in range(H)) >= d[j], "Demand(%s)"%j)
        
    # Setting objective function
    model.setObjective(quicksum(z[h] for h in range(H)), "minimize")
    
    model.data = z
    return model

## Resolving dual reduced master problem
Risolviamo il problema duale tramite gli scarti complementari.

In [10]:
def rmp_dual(rmp:Model, A:list, d:list):
    """Solve the dual problem via complementary slackness
        rmp: the solved primal problem model
        A: the coefficients matrix
        d: the resource vector
    """
    zs = rmp.getVars()
    n = len(d)
    H = len(zs)
    slackness = np.zeros((H+n,n))

    b = [0]*(H+n)
    for h in range(H):
        z = rmp.getVal(zs[h])
        b[h] = z
        for j in range(n):
            slackness[h][j] = A[h][j]*z

    for j in range(n):
        c = -d[j]
        for h in range(H):
            z = rmp.getVal(zs[h])
            c += A[h][j]*z
        slackness[H+j][j] = c
    return linalg.lstsq(slackness,b)[0] 

## Problema di Pricing

\begin{align*}
w^* = \max \sum_{j=1}^n \lambda_j^* x_j\\
\sum_{j=1}^n l_j x_j &\leq L\\
x_j &\in \mathbb{Z}^+ \quad \forall \{1, \ldots, n\}
\end{align*}


In [11]:
def pricing(lambdas:list, l:list, L:int):
    """Generates a pricing problem model
        lambdas: optimal solutions of dual reduced master problem
        l: list of products lengths
        L: maximum product length
    """
    model = Model("Pricing Problem")
    x = {}
    n = len(l)
    
    # Adding the variables to the model
    for j in range(n):
        x[j] = model.addVar(vtype="I", name="x(%s)"%(j))
        model.addCons(x[j] >= 0, "Zero(%s)"%(j))
        
    #Adding demand constraints
    model.addCons(quicksum(l[j]*x[j] for j in range(n)) <= L, "Len(%s)"%j)
        
    # Setting objective function
    model.setObjective(quicksum(lambdas[j]*x[j] for j in range(n)), "maximize")
    
    model.data = x
    return model

## Solving with Column Generation

In [12]:
def cutting_stock_column_generation(l:list, d:list, L:int, m:int, slack:float):
    """Solves the given cutting stock problem via column generation
        l: list of products lengths
        d: list of products demands
        L: maximum paling lenghts
        m: maximum number of palings
        slack: error margin
    """
    # We get the initial cuts patterns matrix
    A = starting_cuts_pattern_set(l, d, L)
    w = 0
    max_iteration = 100
    old_sol = 0
    i = 0
    # Until w is less than one
    while i < max_iteration:
        i+=1
        # We solve the reduced master problem
        rmp = reduced_master_problem(A, d)
        rmp.optimize()
        # We get the dual reduced master problem solution
        lambdas = rmp_dual(rmp, A, d)
        # We solve the pricing problem
        pp = pricing(lambdas, l, L)
        pp.optimize()
        w = pp.getObjVal()
        # If the coefficient w is more than 1
        if w - slack> 1:
           # We add a new column 
            new_column = []
            for a in pp.getVars():
                new_column.append(pp.getVal(a))
            if old_sol != w:
                old_sol = w
                print("Iteration: %s"%i)
                print_results(pp)
            A.append(new_column)
        else:
            # Eureka! We've resolved!
            print("Resolved in %s iterations"%i)
            ext = extended_formulation(A, d)
            ext.optimize()
            print_results(pp)
            print_results(rmp)
            print_results(ext)
            break
    if i==max_iteration:
        print("Unable to find solution in %s iterations"%i)

In [13]:
products = 5
palings_len = 5
palings = 20
lenghts = [random.randint(2, palings_len) for i in range(products)]
demands = [random.randint(2, palings_len) for i in range(products)]

In [14]:
cutting_stock_model = cutting_stock(lenghts, demands, palings_len, palings)

In [15]:
cutting_stock_model.optimize()

In [16]:
print_results(cutting_stock_model)

In model Cutting Stock the optimal solution value is : 9.0


In [17]:
cutting_stock_column_generation(lenghts, demands, palings_len, palings, slack=1e-9)

Iteration: 1
In model Pricing Problem the optimal solution value is : 1.5
Iteration: 3
In model Pricing Problem the optimal solution value is : 1.3888888888888888
Iteration: 4
In model Pricing Problem the optimal solution value is : 1.1951219512195137
Iteration: 5
In model Pricing Problem the optimal solution value is : 1.153846153846154
Resolved in 6 iterations
In model Pricing Problem the optimal solution value is : 1.0000000000000002
In model Reduced Master Problem the optimal solution value is : 9.0
In model Cutting Stock Extended Formulation the optimal solution value is : 9.0
